In [1]:
import numpy as np
import torch

model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```


In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [7]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

X and Y to tensors

In [8]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values.

In [9]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.6295,  0.8457,  0.1010],
        [ 0.8688,  1.4919, -0.2712]], requires_grad=True)
tensor([-0.5647,  0.8586], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [10]:
def model(x):
    return x @ w.t() + b

Predictions

In [11]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 14.4888, 152.5786],
        [ 23.0391, 193.8525],
        [ 63.8551, 260.6313],
        [-24.6704, 143.5958],
        [ 44.2600, 185.0468]], grad_fn=<AddBackward0>)


## Loss function

In [12]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [13]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(6120.7529, grad_fn=<DivBackward0>)


## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases 

In [14]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [15]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.6295,  0.8457,  0.1010],
        [ 0.8688,  1.4919, -0.2712]], requires_grad=True)
tensor([[-4383.1611, -4583.4194, -2906.2979],
        [ 8202.3477,  8346.0996,  5092.6758]])



## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [16]:
w.grad

tensor([[-4383.1611, -4583.4194, -2906.2979],
        [ 8202.3477,  8346.0996,  5092.6758]])

In [17]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (10^-5 in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the learning rate of the algorithm.

We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [18]:
loss = mse(preds, targets)
print(loss)

tensor(6120.7529, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients.

In [19]:
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

## Train the model using gradient descent

we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [20]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [21]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(130.6653, grad_fn=<DivBackward0>)


In [22]:
# Predictions
preds

tensor([[ 56.8944,  73.7796],
        [ 78.8448,  92.4798],
        [126.7730, 146.0092],
        [ 19.7341,  55.2962],
        [ 96.7469,  94.3893]], grad_fn=<AddBackward0>)

In [23]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])